In [3]:
from PrintImport import PrintWebCrawling

In [4]:
PrintWebCrawling()

import requests, bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver_loc = "/Applications/chromedriver"

options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")              # 대부분은 적용이 안 되는 것 같다.
options.add_argument('headless')                           # 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
options.add_argument('disable-gpu')
driver = webdriver.Chrome(driver_loc, options=options)
driver.maximize_window()                                   # 이렇게 해주면 무조건 최대화 시킨다. 위에 픽셀보다 잘 먹힘.
driver.implicitly_wait(3)                                  # 웹페이지 파싱 될 때까지 최대 3초 기다려줌.

from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

from pandas.io.json import json_normalize                  # 그냥 pd.json_normalize를 사용하자.
from lxml import html
from urllib.requ

In [1]:
import requests, bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver_loc = "./chromedriver"

In [2]:
from selenium.webdriver.common.action_chains import ActionChains
import time
sleepTime = 3
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('windows-size = 1920x1080')             # 적용은 안 되지만 headless때 혹시라도 도움될까 싶어서...
# options.add_argument('headless')                           # 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

driver = webdriver.Chrome(driver_loc, options=options)

In [4]:
# 1. 홈페이지로 이동
danawaUrl = 'http://danawa.com'
driver.get(danawaUrl)

In [5]:
# 2. 마우스 오버로 선택이 안 되고 클릭을 해야하기 때문에 분리. 메인 메뉴 클릭.
mainMenu = '//*[@id="danawa_header"]/div[2]/div/div[1]/a/span[2]'
driver.find_element_by_xpath(mainMenu).click()
time.sleep(sleepTime)

In [6]:
# 3. 태블릿.모바일 > 스마트워치/밴드 클릭.
subMenu = '//*[@id="sectionExplodeLayer"]/div[1]/ul/li[3]/a'
targetMenu = '//*[@id="categoryExplodeLayer12"]/div[2]/div[2]/ul/li[1]/a'

sub = driver.find_element_by_xpath(subMenu)
target = driver.find_element_by_xpath(targetMenu)

ActionChains(driver).move_to_element(sub).click(target).perform()
time.sleep(sleepTime)

In [17]:
# 4. 현재 페이지 소스 가져오기
html = driver.page_source.encode('utf-8')

bs = bs4.BeautifulSoup(html, 'html.parser')
# bs

In [22]:
# 상품명 구하기
name = bs.select('div > div.prod_info > p > a')
# name

In [57]:
name[15].text.strip()    # 1 ~ 30 번까지 홈페이지 상품과 일치. 0번에는 14번 상품이 상품명이 조금 다르게 중복으로 들어가있음. (len(name) = 31)

'삼성전자 갤럭시 워치 46mm'

In [58]:
# 평점 구하기 / 별이 100분위로 나뉘어져있음. 4개 4.5개 5개 이런식이 아니라 점수가 93점이면 실제로 별 5칸을 100%로 두고 93%가 색칠된거임. 따라서 점수를 취득.
rate = bs.select('div > div.prod_info > div.prod_sub_info > div > dl.meta_item.mt_comment > dd > div > div.point_num')
# rate    # 마지막 30번 애플워치가 평점이나 상품평이 없어서 len 결과가 30으로 나옴. 실제로 0번은 의미 없는 데이터고, 1번부터 유효 데이터. 1 ~ 29번 상품과 일치. 30번은 데이터 없음.

In [72]:
rate[1].text.replace('점','')

'93'

In [59]:
# 상품평 건수 구하기
review = bs.select('div > div.prod_info > div.prod_sub_info > div > dl.meta_item.mt_comment > dd > a > strong')
# review    # 마지막 30번 애플워치가 평점이나 상품평이 없어서 len 결과가 30으로 나옴. 실제로 0번은 의미 없는 데이터고, 1번부터 유효 데이터. 1 ~ 29번 상품과 일치. 30번은 데이터 없음

In [63]:
review[1].text.replace(',','')    # 천 단위 구분 기호 삭제처리.

'1274'

In [110]:
resultList = []

nameLen = len(name)
for i in range(1,nameLen):    # 0번에 리스트에 존재하지 않는 상품이 있음.
    try:
        eachName = name[i].text.strip()    # 상품명.
    except Exception as e:
        pass
    try:
        eachRate = rate[i].text.replace('점','')    # 평점.
    except Exception as e:
        pass
    try:
        eachReview = review[i].text.replace(',','')    # 상품평 건수.
    except Exception as e:
        pass
    
    resultList.append([eachName, eachRate, eachReview])
    eachName = eachRate = eachReview = None    # None을 넣어야 맞는걸까? ''으로 넣어야 맞는걸까? None는 아예 값이 안 들어가고, '' 은 문자열이긴 한데 아무것도 없는 문자열로 들어간다.

resultList    

[['샤오미 미 밴드4', '93', '1274'],
 ['삼성전자 갤럭시 워치 액티브2 40mm 알루미늄 케이스', '97', '473'],
 ['삼성전자 갤럭시 워치 액티브2 44mm 스테인리스 스틸 케이스', '97', '330'],
 ['샤오미 화미 어메이즈핏 GTR 47mm', '94', '116'],
 ['삼성전자 갤럭시 워치 액티브', '96', '856'],
 ['샤오미 화미 어메이즈핏 GTS', '96', '242'],
 ['삼성전자 갤럭시 워치 액티브2 44mm 알루미늄 케이스', '96', '234'],
 ['삼성전자 갤럭시 핏', '94', '540'],
 ['APPLE 워치 시리즈5 44mm 스페이스 그레이 알루미늄 케이스', '98', '129'],
 ['샤오미 헬로우 스마트워치', '55', '106'],
 ['삼성전자 갤럭시 핏e', '89', '609'],
 ['삼성전자 갤럭시 워치 액티브2 40mm 스테인리스 스틸 케이스', '94', '120'],
 ['삼성전자 갤럭시 워치 액티브2 44mm 골프 에디션', '99', '129'],
 ['화웨이 워치 GT 2 46mm 클래식', '94', '84'],
 ['삼성전자 갤럭시 워치 46mm', '95', '1033'],
 ['APPLE 워치 시리즈5 44mm 실버 알루미늄 케이스', '98', '55'],
 ['화웨이 밴드 4', '85', '71'],
 ['APPLE 워치 시리즈5 40mm 스페이스 그레이 알루미늄 케이스', '95', '250'],
 ['APPLE 워치 시리즈5 40mm 실버 알루미늄 케이스', '95', '214'],
 ['샤오미 화미 어메이즈핏 빕 라이트', '90', '172'],
 ['프라임원 엑스핏 GTS', '76', '25'],
 ['APPLE 워치 시리즈5 나이키+ 44mm 스페이스 그레이 알루미늄 케이스', '92', '14'],
 ['핏빗 Charge 4', '96', '5'],
 ['APPLE 워치 시리즈5 44mm Cellular 스페이스 

In [14]:
j = 0
while(True):
    try:
        if ((j+1) % 10) == 0:      # 간격이 10이니까... 11 21 31 41
            nextPage = driver.find_element_by_xpath('//*[@class="prod_num_nav"]/div/a')
            ActionChains(driver).move_to_element(nextPage).click().perform()
        else:
            nextPage = driver.find_element_by_xpath('//*[@class="prod_num_nav"]/div/div/a[{}]'.format(j%10+1))
            ActionChains(driver).move_to_element(nextPage).click().perform()
        time.sleep(3)
    except Exception as e:
        break
    j = j + 1

In [23]:
import requests, bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver_loc = "./chromedriver"

from selenium.webdriver.common.action_chains import ActionChains
import time
sleepTime = 1
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

options = webdriver.ChromeOptions()
options.add_argument('windows-size = 1920x1080')             # 적용은 안 되지만 headless때 혹시라도 도움될까 싶어서...
# options.add_argument('headless')                           # 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

driver = webdriver.Chrome(driver_loc, options=options)

# 1. 홈페이지로 이동
# danawaUrl = 'http://danawa.com'    # 랜덤한 시간에 광고가 떠서 확률적으로 메인 페이지에서 마우스가 이동해가며 누르면 광고를 누르고 들어가버린다...
danawaUrl = 'http://prod.danawa.com/list/?cate=12228647'
driver.get(danawaUrl)

# # 2. 마우스 오버로 선택이 안 되고 클릭을 해야하기 때문에 분리. 메인 메뉴 클릭.
# mainMenu = '//*[@id="danawa_header"]/div[2]/div/div[1]/a/span[2]'
# driver.find_element_by_xpath(mainMenu).click()
# time.sleep(sleepTime)

# # 3. 태블릿.모바일 > 스마트워치/밴드 클릭.
# subMenu = '//*[@id="sectionExplodeLayer"]/div[1]/ul/li[3]/a'
# targetMenu = '//*[@id="categoryExplodeLayer12"]/div[2]/div[2]/ul/li[1]/a'

# sub = driver.find_element_by_xpath(subMenu)
# target = driver.find_element_by_xpath(targetMenu)

# ActionChains(driver).move_to_element(sub).click(target).perform()
# time.sleep(sleepTime)

resultList = []
j = 1    # 페이지 정보.
while(True):
        
    # 4. 현재 페이지 소스 가져오기.
    html = driver.page_source.encode('utf-8')

    bs = bs4.BeautifulSoup(html, 'html.parser')
    # bs
    
    # 상품명 구하기
    name = bs.select('div > div.prod_info > p > a')
    
    # 평점 구하기 / 별이 100분위로 나뉘어져있음. 4개 4.5개 5개 이런식이 아니라 점수가 93점이면 실제로 별 5칸을 100%로 두고 93%가 색칠된거임. 따라서 점수를 취득.
    rate = bs.select('div > div.prod_info > div.prod_sub_info > div > dl.meta_item.mt_comment > dd > div > div.point_num')
    # rate    # 마지막 30번 애플워치가 평점이나 상품평이 없어서 len 결과가 30으로 나옴. 실제로 0번은 의미 없는 데이터고, 1번부터 유효 데이터. 1 ~ 29번 상품과 일치. 30번은 데이터 없음.
    
    # 상품평 건수 구하기
    review = bs.select('div > div.prod_info > div.prod_sub_info > div > dl.meta_item.mt_comment > dd > a > strong')

    nameLen = len(name)
    for i in range(0,nameLen):    # 0번에 리스트에 존재하지 않는 상품이 있음. (1페이지 한정) 에드리더가 뜨면 0번 상품이 잡히고 안 뜨면 이상하게 잡힌다...
        if j == 1:
            if i == 0:
                continue          # 1페이지일 때 0번 리스트면 반복문을 다음으로 넘겨라.

        # 상품명은 비어있는 데이터가 없지만, 평점이나 상품평 건수는 비어있는 데이터가 있다. 하지만 혹시 몰라 상품명도 모두 예외처리 함.
        try:
            eachName = name[i].text.strip()    # 상품명.
        except Exception as e:
            pass
        try:
            eachRate = rate[i].text.replace('점','')    # 평점.
        except Exception as e:
            pass
        try:
            eachReview = review[i].text.replace(',','')    # 상품평 건수.
        except Exception as e:
            pass
        

        resultList.append([eachName, eachRate, eachReview])
        eachName = eachRate = eachReview = None    # None을 넣어야 맞는걸까? ''으로 넣어야 맞는걸까? None는 아예 값이 안 들어가고, '' 은 문자열이긴 한데 아무것도 없는 문자열로 들어간다.

    # 5. 반복문 페이지 넘기기.
    try:
        if ((j+1) % 10) == 0:      # 간격이 10이니까... 11 21 31 41
            nextPage = driver.find_element_by_xpath('//*[@class="prod_num_nav"]/div/a')
            ActionChains(driver).move_to_element(nextPage).click().perform()
        else:
            nextPage = driver.find_element_by_xpath('//*[@class="prod_num_nav"]/div/div/a[{}]'.format(j%10+1))
            ActionChains(driver).move_to_element(nextPage).click().perform()
        time.sleep(sleepTime)
    except Exception as e:
        print(e)
        break    
        
    j = j + 1    # 페이지 넘기기용 변수.

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="prod_num_nav"]/div/div/a[2]"}
  (Session info: chrome=81.0.4044.138)



In [24]:
import pandas as pd

resultDF = pd.DataFrame(resultList, columns=['상품명','평점','평가수'])
resultDF

,상품명,평점,평가수
0,샤오미 미 밴드4,93,1274
1,삼성전자 갤럭시 워치 액티브2 40mm 알루미늄 케이스,97,473
2,삼성전자 갤럭시 워치 액티브2 44mm 스테인리스 스틸 케이스,97,330
3,샤오미 화미 어메이즈핏 GTR 47mm,94,116
4,삼성전자 갤럭시 워치 액티브,96,856
...,...,...,...
317,APPLE 워치 시리즈4 나이키+ 44mm Cellular 알루미늄 케이스,None,None
318,APPLE 워치 시리즈3 나이키+ 42mm 알루미늄 케이스,None,None
319,레노버 HW01,None,None
320,유니콘 SH-700T,None,None


In [25]:
resultDF.to_csv('test.csv', index=False)

### 마무리

In [40]:
import requests, bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver_loc = "./chromedriver"

from selenium.webdriver.common.action_chains import ActionChains
import time
sleepTime = 1
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

options = webdriver.ChromeOptions()
options.add_argument('windows-size = 1920x1080')             # 적용은 안 되지만 headless때 혹시라도 도움될까 싶어서...
# options.add_argument('headless')                           # 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

driver = webdriver.Chrome(driver_loc, options=options)

# 1. 홈페이지로 이동
# danawaUrl = 'http://danawa.com'    # 랜덤한 시간에 광고가 랜덤한 위치로 떠서 확률적으로 메인 페이지에서 마우스가 이동해가며 누르면 광고를 누르고 들어가버린다...
                                     # 특히 크롤링을 반복하면 해당 브라우저에서는 확률이 더 올라가는 것 같음.
danawaUrl = 'http://prod.danawa.com/list/?cate=12228647'
driver.get(danawaUrl)

# # 2. 마우스 오버로 선택이 안 되고 클릭을 해야하기 때문에 분리. 메인 메뉴 클릭.
# mainMenu = '//*[@id="danawa_header"]/div[2]/div/div[1]/a/span[2]'
# driver.find_element_by_xpath(mainMenu).click()
# time.sleep(sleepTime)

# # 3. 태블릿.모바일 > 스마트워치/밴드 클릭.
# subMenu = '//*[@id="sectionExplodeLayer"]/div[1]/ul/li[3]/a'
# targetMenu = '//*[@id="categoryExplodeLayer12"]/div[2]/div[2]/ul/li[1]/a'

# sub = driver.find_element_by_xpath(subMenu)
# target = driver.find_element_by_xpath(targetMenu)

# ActionChains(driver).move_to_element(sub).click(target).perform()
# time.sleep(sleepTime)

resultList = []
j = 1    # 페이지 정보.
while(True):
        
    # 4. 현재 페이지 소스 가져오기.
    html = driver.page_source.encode('utf-8')

    bs = bs4.BeautifulSoup(html, 'html.parser')
    # bs
    
    # 상품명 구하기
    name = bs.select('div > div.prod_info > p > a')
    
    # 평점 구하기 / 별이 100분위로 나뉘어져있음. 4개 4.5개 5개 이런식이 아니라 점수가 93점이면 실제로 별 5칸을 100%로 두고 93%가 색칠된거임. 따라서 점수를 취득.
    rate = bs.select('div > div.prod_info > div.prod_sub_info > div > dl.meta_item.mt_comment > dd > div > div.point_num')
    # rate    # 마지막 30번 애플워치가 평점이나 상품평이 없어서 len 결과가 30으로 나옴. 실제로 0번은 의미 없는 데이터고, 1번부터 유효 데이터. 1 ~ 29번 상품과 일치. 30번은 데이터 없음.
    
    # 상품평 건수 구하기
    review = bs.select('div > div.prod_info > div.prod_sub_info > div > dl.meta_item.mt_comment > dd > a > strong')

    nameLen = len(name)
    for i in range(0,nameLen):    # 0번에 리스트에 존재하지 않는 상품이 있음. (1페이지 한정) 에드리더가 뜨면 0번 상품이 잡히고 안 뜨면 이상하게 잡힌다...
        if j == 1:
            if i == 0:
                continue          # 1페이지일 때 0번 리스트면 반복문을 다음으로 넘겨라.

        # 상품명은 비어있는 데이터가 없지만, 평점이나 상품평 건수는 비어있는 데이터가 있다. 하지만 혹시 몰라 상품명도 모두 예외처리 함.
        try:
            eachName = name[i].text.strip()    # 상품명.
        except Exception as e:
            pass
        try:
            eachRate = rate[i].text.replace('점','')    # 평점.
        except Exception as e:
            pass
        try:
            eachReview = review[i].text.replace(',','')    # 상품평 건수.
        except Exception as e:
            pass
        

        resultList.append([eachName, eachRate, eachReview])
#         eachName = eachRate = eachReview = None    # None을 넣어야 맞는걸까? ''으로 넣어야 맞는걸까? None는 아예 값이 안 들어가고, '' 은 문자열이긴 한데 아무것도 없는 문자열로 들어간다.
        eachName = eachRate = eachReview = ''
        
    # 5. 반복문 페이지 넘기기.
    try:
        if ((j+1) % 10) == 0:      # 간격이 10이니까... 11 21 31 41
            nextPage = driver.find_element_by_xpath('//*[@class="prod_num_nav"]/div/a')
            ActionChains(driver).move_to_element(nextPage).click().perform()
        else:
            nextPage = driver.find_element_by_xpath('//*[@class="prod_num_nav"]/div/div/a[{}]'.format(j%10+1))
            ActionChains(driver).move_to_element(nextPage).click().perform()
        time.sleep(sleepTime)
    except Exception as e:
        print(e)
        break    
        
    j = j + 1    # 페이지 넘기기용 변수.

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="prod_num_nav"]/div/div/a[2]"}
  (Session info: chrome=81.0.4044.138)



In [41]:
import pandas as pd

resultDF = pd.DataFrame(resultList, columns=['상품명','평점','평가수'])
resultDF

,상품명,평점,평가수
0,샤오미 미 밴드4,93,1274
1,삼성전자 갤럭시 워치 액티브2 40mm 알루미늄 케이스,97,473
2,삼성전자 갤럭시 워치 액티브2 44mm 스테인리스 스틸 케이스,97,330
3,샤오미 화미 어메이즈핏 GTR 47mm,94,116
4,삼성전자 갤럭시 워치 액티브,96,856
...,...,...,...
317,APPLE 워치 시리즈4 나이키+ 44mm Cellular 알루미늄 케이스,,
318,APPLE 워치 시리즈3 나이키+ 42mm 알루미늄 케이스,,
319,레노버 HW01,,
320,유니콘 SH-700T,,


In [42]:
resultDF.to_csv('smartWatchUserRate.csv', index=False)

In [43]:
resultDF['상품명']

0                                      샤오미 미 밴드4
1                 삼성전자 갤럭시 워치 액티브2 40mm 알루미늄 케이스
2             삼성전자 갤럭시 워치 액티브2 44mm 스테인리스 스틸 케이스
3                          샤오미 화미 어메이즈핏 GTR 47mm
4                                삼성전자 갤럭시 워치 액티브
                         ...                    
317    APPLE 워치 시리즈4 나이키+ 44mm Cellular 알루미늄 케이스
318             APPLE 워치 시리즈3 나이키+ 42mm 알루미늄 케이스
319                                     레노버 HW01
320                                  유니콘 SH-700T
321                                  삼성전자 기어S 3G
Name: 상품명, Length: 322, dtype: object

In [44]:
# 아마존은...... 상품명을..... 워치가 애플처럼 개별 코드명이 있지 않으면 모든 데이터를 수작업으로 비교해서 매칭시켜줘야한다... 한글 상품명이라...효율이 떨어질 듯

In [45]:
resultDF.loc[:,['상품명']].drop_duplicates()    # 중복 제거 해봤자 몇 개 안 없어진다.

,상품명
0,샤오미 미 밴드4
1,삼성전자 갤럭시 워치 액티브2 40mm 알루미늄 케이스
2,삼성전자 갤럭시 워치 액티브2 44mm 스테인리스 스틸 케이스
3,샤오미 화미 어메이즈핏 GTR 47mm
4,삼성전자 갤럭시 워치 액티브
...,...
291,APPLE 워치 시리즈4 나이키+ 44mm Cellular 알루미늄 케이스
292,APPLE 워치 시리즈3 나이키+ 42mm 알루미늄 케이스
293,레노버 HW01
294,유니콘 SH-700T


In [46]:
# 모델명으로 조회를 해보려 하였으나... 커뮤니티에서 그에 해당하는 정확한 데이터 정보와 매칭을 시키려면...
# 상품명의 띄워쓰기, 오타, 애칭, 한글 영어 섞어 쓰기 등 모든 것들을 분석해야한다... 이건... 수작업 처리가 아니라 빅데이터 학습으로 가야할 것 같은데?

In [47]:
# 에누리쪽 평점이랑 합쳐보고 싶은데... 상품명이 다르다.....

In [ ]:
resultDF.loc[resultDF.평가수.astype(int) > 200]   # 상품평 수가 많은 것들을 조회해보려 했는데 처리가 안 되네... 반복문 돌려야 할 듯...

In [1]:
import requests, bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver_loc = "./chromedriver"

from selenium.webdriver.common.action_chains import ActionChains
import time
sleepTime = 1
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

options = webdriver.ChromeOptions()
options.add_argument('windows-size = 1920x1080')             # 적용은 안 되지만 headless때 혹시라도 도움될까 싶어서...
# options.add_argument('headless')                           # 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

driver = webdriver.Chrome(driver_loc, options=options)

# 1. 홈페이지로 이동
# danawaUrl = 'http://danawa.com'    # 랜덤한 시간에 광고가 랜덤한 위치로 떠서 확률적으로 메인 페이지에서 마우스가 이동해가며 누르면 광고를 누르고 들어가버린다...
danawaUrl = 'http://prod.danawa.com/list/?cate=12228647'
driver.get(danawaUrl)

In [2]:
target = '//*[@id="dlMaker_simple"]/dd/div[2]/button[1]'
driver.find_element_by_xpath(target).click()
time.sleep(sleepTime)

In [3]:
html = driver.page_source.encode('utf-8')
bs = bs4.BeautifulSoup(html, 'html.parser')

### 수업 때 배운 범위 좁히기 하위분류로는 분류가 안 되고 에러가 발생함.

In [4]:
ulTags = bs.find_all(name = 'ul', attrs = {'id':'selectMaker_simple_priceCompare_A'})
# ulTags

In [5]:
labelTags = ulTags.find_all(name = 'label')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

### 그래서 바로 label을 찾았더니 상세 검색이 아닌 상단의 5개 제조사만 나타남.

In [6]:
manufac = bs.find_all(name = 'label')

In [7]:
manufac[1].name

'label'

In [8]:
manufac[1].text.strip()

'샤오미'

In [9]:
manufac[2].attrs

{'title': 'APPLE'}

In [10]:
manufac[4].attrs['title']    # 이건 더보기 하기 전 위에꺼만...

'화웨이'

### 그래서 다른 방법 시도

In [11]:
manufacturer = bs.select('#selectMaker_simple_priceCompare_A > li:nth-child(1) > label')
manufacturer

[<label title="나이키"> <input data-attribute-name="제조사" id="searchMaker27244" name="searchMaker[]" onclick="javascript:equalCheckSearchOption(this, 'searchMaker', ''); getProductList(1);" type="checkbox" value="27244"/> 나이키	</label>]

In [12]:
manufacturer = bs.select('#selectMaker_simple_priceCompare_A > li:nth-child(2) > label')
manufacturer

[<label title="네이버랩스"> <input data-attribute-name="제조사" id="searchMaker115337" name="searchMaker[]" onclick="javascript:equalCheckSearchOption(this, 'searchMaker', ''); getProductList(1);" type="checkbox" value="115337"/> 네이버랩스	</label>]

In [13]:
manufacturer[0].text.strip()    # 이 방법은 child() 안의 숫자가 달라져 저기다 반복문을 돌리고 manufacturer[0]은 0번으로 고정.

'네이버랩스'

In [14]:
manufacturer[0].attrs

{'title': '네이버랩스'}

In [ ]:
manufacturer = []
k = 1
while(True):
    try:
        eachTags = bs.select('#selectMaker_simple_priceCompare_A > li:nth-child({}) > label'.format(k))
        eachFac = eachTags[0].attrs['title']
        manufacturer.append(eachFac)
    except Exception as e:
        break
    k = k + 1
        
# manufacturer

In [ ]:
manufacturer = []
for k in range(1,63):
    eachTags = bs.select('#selectMaker_simple_priceCompare_A > li:nth-child({}) > label'.format(k))
    eachFac = eachTags[0].text.strip()
    manufacturer.append(eachFac)
    
# manufacturer

In [ ]:
# # 2. 마우스 오버로 선택이 안 되고 클릭을 해야하기 때문에 분리. 메인 메뉴 클릭.
# mainMenu = '//*[@id="danawa_header"]/div[2]/div/div[1]/a/span[2]'
# driver.find_element_by_xpath(mainMenu).click()
# time.sleep(sleepTime)

# # 3. 태블릿.모바일 > 스마트워치/밴드 클릭.
# subMenu = '//*[@id="sectionExplodeLayer"]/div[1]/ul/li[3]/a'
# targetMenu = '//*[@id="categoryExplodeLayer12"]/div[2]/div[2]/ul/li[1]/a'

# sub = driver.find_element_by_xpath(subMenu)
# target = driver.find_element_by_xpath(targetMenu)

# ActionChains(driver).move_to_element(sub).click(target).perform()
# time.sleep(sleepTime)

In [ ]:
# 1 페이지    크롬 XPath 복사 기능을 이용했을 때 언제는 div[4]고 언제는 div[5]고 확률적으로 변한다;;
#productListArea > div.prod_num_nav > div > div > a.num.now_on
//*[@id="productListArea"]/div[4]/div/div/a[1]    # // 찾아라 * 아무거나 id=productListArea인 것[]로 문자열로 묶임 / 그 아래에 div 4번째에 / div 아래에 / div 아래에 / a 1번째
//*[@id="productListArea"]/div[5]/div/div/a[2]

In [ ]:
# 2 페이지
#productListArea > div.prod_num_nav > div > div > a:nth-child(2)
//*[@id="productListArea"]/div[4]/div/div/a[2]
//*[@id="productListArea"]/div[4]/div/div/a[2]

In [ ]:
# 10 페이지
#productListArea > div.prod_num_nav > div > div > a:nth-child(10)
//*[@id="productListArea"]/div[4]/div/div/a[10]

In [ ]:
# 페이지 넘김
#productListArea > div.prod_num_nav > div > a
//*[@id="productListArea"]/div[4]/div/a

In [ ]:
# 11 페이지(마지막 페이지)
#productListArea > div.prod_num_nav > div > div > a
//*[@id="productListArea"]/div[4]/div/div/a

In [9]:
for j in range(20):
    print(j%10+1)

1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
